In [19]:
val r = sc.cassandraTable("retail","receipts_by_credit_card")

In [20]:
r.first

com.datastax.spark.connector.CassandraRow = CassandraRow{credit_card_number: 346314966079541, receipt_timestamp: 2015-02-01 21:39:14+0000, receipt_id: 1444719922743, credit_card_type: American Express, customer: {first_name: Alice, last_name: Briggs, addr1: 647 Clarence Drv, city: Lulaton, state: IN, zip: 68971}, receipt_total: 1249.8700000000001, store_id: 46}

In [21]:
case class Cust(first_name: String, last_name: String, zip:String)

In [22]:
def udt2Cust(cust:UDTValue) = Cust(cust.getString("first_name"), cust.getString("last_name"), cust.getString("zip"))

In [29]:
val mapped = r.map( row =>  ( (row.getInt("store_id"), row.getUDTValue("customer")), row.getDecimal("receipt_total") ) )

In [28]:
mapped.collect take 10 foreach println

((46,{first_name: Alice, last_name: Briggs, addr1: 647 Clarence Drv, city: Lulaton, state: IN, zip: 68971}),1249.8700000000001)
((374,{first_name: Corey, last_name: Black, addr1: 432 Aspen Road, city: Cogdell, state: CA, zip: 45482}),1589.8500000000001)
((350,{first_name: Katherine, last_name: Puckett, addr1: 430 Narrows Trail, city: Mayday, state: AZ, zip: 65623}),119.94999999999999)
((169,{first_name: Karen, last_name: Vazquez, addr1: 1049 Scholl Park, city: Cave Spring, state: WY, zip: 39835}),359.96)
((322,{first_name: Kathryn, last_name: Fuller, addr1: 930 Marne Street, city: Surrency, state: OK, zip: 22505}),1169.8400000000001)
((139,{first_name: Tasha, last_name: Tyler, addr1: 509 Prame Terrace, city: Axson, state: KY, zip: 52582}),989.77)
((106,{first_name: Tim, last_name: Joyner, addr1: 1477 Eastview Circle, city: Homer, state: MD, zip: 38177}),399.95)
((247,{first_name: Brooklyn, last_name: Gilliam, addr1: 443 Thompson Drv, city: Sunbury, state: HI, zip: 38177}),1069.76)
((31

In [33]:
val storeCustAmt = mapped.reduceByKey(_+_)

In [34]:
storeCustAmt.collect take 10 foreach println

((88,{first_name: Paige, last_name: Chaney, addr1: 431 Home St, city: Sessoms, state: NC, zip: 21690}),919.86)
((329,{first_name: Alicia, last_name: Adkins, addr1: 1420 Blaine Road, city: Thunderbolt, state: ME, zip: 55679}),49.99)
((251,{first_name: Abigail, last_name: Charles, addr1: 750 Stacy Terrace, city: Barwick, state: UT, zip: 22241}),1039.8799999999999)
((217,{first_name: Christine, last_name: Garza, addr1: 1661 Winfield Ave, city: East Point, state: ID, zip: 64403}),329.95)
((102,{first_name: Katelyn, last_name: Kent, addr1: 1007 Hickam Trail, city: Poulan, state: CO, zip: 45579}),559.9)
((283,{first_name: Vicky, last_name: Morales, addr1: 1452 Dana Trail, city: Springfield, state: WA, zip: 63003}),1939.71)
((362,{first_name: Linda, last_name: Underwood, addr1: 957 Middleton Park, city: Kingsland, state: UT, zip: 65870}),1189.8600000000001)
((245,{first_name: Jacob, last_name: George, addr1: 1174 Sharonwood Run, city: Montgomery, state: NV, zip: 09480}),1359.8)
((320,{first_n

In [38]:
val storeCustAmt2 = storeCustAmt.map{ case ((store, cust ), amt) => (store, (amt,cust)) }

In [36]:
storeCustAmt2.collect take 10 foreach println

(88,(919.86,{first_name: Paige, last_name: Chaney, addr1: 431 Home St, city: Sessoms, state: NC, zip: 21690}))
(329,(49.99,{first_name: Alicia, last_name: Adkins, addr1: 1420 Blaine Road, city: Thunderbolt, state: ME, zip: 55679}))
(251,(1039.8799999999999,{first_name: Abigail, last_name: Charles, addr1: 750 Stacy Terrace, city: Barwick, state: UT, zip: 22241}))
(217,(329.95,{first_name: Christine, last_name: Garza, addr1: 1661 Winfield Ave, city: East Point, state: ID, zip: 64403}))
(102,(559.9,{first_name: Katelyn, last_name: Kent, addr1: 1007 Hickam Trail, city: Poulan, state: CO, zip: 45579}))
(283,(1939.71,{first_name: Vicky, last_name: Morales, addr1: 1452 Dana Trail, city: Springfield, state: WA, zip: 63003}))
(362,(1189.8600000000001,{first_name: Linda, last_name: Underwood, addr1: 957 Middleton Park, city: Kingsland, state: UT, zip: 65870}))
(245,(1359.8,{first_name: Jacob, last_name: George, addr1: 1174 Sharonwood Run, city: Montgomery, state: NV, zip: 09480}))
(320,(479.9399

In [46]:
val top10customersperstore = storeCustAmt2.groupByKey.flatMap{ case (store, custAmtList) => custAmtList.toArray.sortBy( ca => -ca._1 ).take(10).map(x => (store, x))}

In [50]:
top10customersperstore.collect take 20 foreach println


(68,(2769.73,{first_name: Jake, last_name: Russo, addr1: 1564 Browns Blvd, city: Nankin, state: ID, zip: 45259}))
(68,(2189.72,{first_name: Kylie, last_name: Potter, addr1: 1585 Gantz Terrace, city: Wray, state: NC, zip: 10205}))
(68,(1979.78,{first_name: Mason, last_name: Maldonado, addr1: 787 Stout Ln, city: Appling, state: KS, zip: 86855}))
(68,(1889.82,{first_name: Randy, last_name: Boone, addr1: 1404 Daisy Way, city: Helena, state: WV, zip: 15978}))
(68,(1779.77,{first_name: Steven, last_name: Hines, addr1: 1572 Lenox Trail, city: Harrietts Bluff, state: CO, zip: 45274}))
(68,(1749.76,{first_name: Kim, last_name: Bass, addr1: 567 Gray Road, city: Danielsville, state: CT, zip: 78262}))
(68,(1709.81,{first_name: Arthur, last_name: Meadows, addr1: 1482 Limestone Rd, city: Dixie, state: OK, zip: 76765}))
(68,(1619.79,{first_name: Scott, last_name: Marsh, addr1: 1717 Liberty Road, city: Soperton, state: MN, zip: 26848}))
(68,(1599.84,{first_name: Cindy, last_name: Bartlett, addr1: 1253

In [57]:
val top10customersperstore_flat= top10customersperstore.map{ case (store,(amt,cust) ) => (store, amt, cust) }
top10customersperstore_flat.take(10).foreach(println)


(68,2769.73,{first_name: Jake, last_name: Russo, addr1: 1564 Browns Blvd, city: Nankin, state: ID, zip: 45259})
(68,2189.72,{first_name: Kylie, last_name: Potter, addr1: 1585 Gantz Terrace, city: Wray, state: NC, zip: 10205})
(68,1979.78,{first_name: Mason, last_name: Maldonado, addr1: 787 Stout Ln, city: Appling, state: KS, zip: 86855})
(68,1889.82,{first_name: Randy, last_name: Boone, addr1: 1404 Daisy Way, city: Helena, state: WV, zip: 15978})
(68,1779.77,{first_name: Steven, last_name: Hines, addr1: 1572 Lenox Trail, city: Harrietts Bluff, state: CO, zip: 45274})
(68,1749.76,{first_name: Kim, last_name: Bass, addr1: 567 Gray Road, city: Danielsville, state: CT, zip: 78262})
(68,1709.81,{first_name: Arthur, last_name: Meadows, addr1: 1482 Limestone Rd, city: Dixie, state: OK, zip: 76765})
(68,1619.79,{first_name: Scott, last_name: Marsh, addr1: 1717 Liberty Road, city: Soperton, state: MN, zip: 26848})
(68,1599.84,{first_name: Cindy, last_name: Bartlett, addr1: 1253 Clover Street, c

In [58]:
top10customersperstore_flat.saveToCassandra("retail","top_customers_by_store")